In [2]:
import numpy as np

In [29]:
data_dict = {}
for i in range(6):
    data_dict[i] = np.arange(i * 2688, (i + 1) * 2688 - 269)

In [37]:
idxs = np.zeros(1614)
for i in range(6):
    idxs[i * 269: (i + 1) * 269] = np.arange((i + 1) * 2688 - 269, (i + 1) * 2688)

In [39]:
idxs.astype(int)

array([ 2419,  2420,  2421, ..., 16125, 16126, 16127])

In [52]:
import torch
(abs(torch.tensor([0, 1]) - torch.tensor([1, 3]))).mean()

RuntimeError: mean(): could not infer output dtype. Input dtype must be either a floating point or complex dtype. Got: Long

In [56]:
torch.tensor([[137.4103], [ 93.9909]]).squeeze() - torch.tensor([67., 72.])

tensor([70.4103, 21.9909])

In [20]:
import pandas as pd

df = pd.read_csv('./data/dataloader.csv', header=None)

In [57]:
len(df)

16128

In [8]:
2688 - 269

2419

In [15]:
269 * 6

1614

In [4]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file, header=None)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Extract 11 input values and 1 target value
        inputs = self.data.iloc[idx, :-1].values.astype(np.float32)
        target = self.data.iloc[idx, -1].astype(np.float32)

        return inputs, target

In [5]:
import torch
import torch.nn as nn

class RegressionModel(nn.Module):
    def __init__(self, input_size):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.fc3(x)
        return x

In [6]:
from tqdm import tqdm

def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()

    for epoch in tqdm(range(num_epochs)):
        for inputs, target in train_loader:
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, target.unsqueeze(1))  # Ensure target has the right shape

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

In [9]:
def test_model(model, test_loader, criterion):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for inputs, target in test_loader:

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, target.unsqueeze(1))  # Ensure target has the right shape

            total_loss += loss.item()

    avg_loss = total_loss / len(test_loader)
    print(f'Test Loss: {avg_loss}')


In [10]:
from sklearn.model_selection import train_test_split
import torch.optim as optim

model = RegressionModel(11)
criterion = nn.MSELoss()

dataset = CustomDataset('./data/dataloader.csv')
batch_size = 32
print('splitting data')
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

print('loading in data')
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
print('loading in test data')
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

optimizer = optim.Adam(model.parameters(), lr=0.001)
# Train the model
print('start training')
train_model(model, train_loader, criterion, optimizer, num_epochs=10)

# Test the model
test_model(model, test_loader, criterion)

splitting data
loading in data
loading in test data
start training


 10%|█         | 1/10 [00:00<00:01,  5.30it/s]

Epoch 1/10, Loss: 39.493099212646484


 20%|██        | 2/10 [00:00<00:01,  5.52it/s]

Epoch 2/10, Loss: 124.55828857421875


...

 90%|█████████ | 9/10 [00:01<00:00,  5.21it/s]

Epoch 9/10, Loss: 35.155879974365234


100%|██████████| 10/10 [00:01<00:00,  5.27it/s]

Epoch 10/10, Loss: 77.2245864868164


Test Loss: 56.593664471465765


In [62]:
input_tensor = [32.59310344827586,32.59310344827586,89.0206896551724,67.1103448275862,508.96666666666664,462.58620689655174,1262.5217391304348,40.47543859649122,68.05172413793103,32.59310344827586,32.59310344827586]

input_tensor = torch.tensor(input_tensor, dtype=torch.float32)
model.eval()
with torch.no_grad():
    prediction = model(input_tensor)
predicted_value = prediction.item()

In [63]:
predicted_value

61.749935150146484

In [64]:
67.0 - predicted_value

5.250064849853516

In [65]:
input_tensor = [49.88793103448276,49.88793103448276,90.78793103448275,66.5844827586207,817.3315789473685,699.903448275862,1301.1739130434783,32.94736842105263,68.19137931034483,49.88793103448276,49.88793103448276]
input_tensor = torch.tensor(input_tensor, dtype=torch.float32)
model.eval()
with torch.no_grad():
    prediction = model(input_tensor)
predicted_value = prediction.item()